In [1]:
!pip -q install -U transformers accelerate datasets sentencepiece

In [2]:
import torch, time, json, os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device, "|CUDA:", torch.version.cuda if torch.cuda.is_available() else "N/A")

Device: cuda |CUDA: 12.6


In [18]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
alternative_model_id = "microsoft/phi-2"

fallback_model_id = "distilgpt2"

def load_model(model_name):
  try:
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    mdl = AutoModelForCausalLM.from_pretrained(
        model_name,
        dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    return tok, mdl, model_name
  except Exception as e:
    print(f"Primary model failed: {e} \nFalling back to {fallback_model_id}...")
    tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
    mdl = AutoModelForCausalLM.from_pretrained(
        fallback_model_id,
        dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded model:", active_model_id)

Loaded model: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [19]:
gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=0 if device == "cuda" else -1
)

prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
out = gen(
    prompt,
    max_new_tokens=120,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)[0]["generated_text"]

print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.


# [TinyLlama/TinyLlama-1.1B-Chat-v1.0]
Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.

A Knowledge Graph is a sophisticated system that enables the integration of various sources of data from different sources into a single, comprehensive database of medical knowledge. It includes information from various sources such as medical journals, patient records, and other health-related websites. The system uses machine learning algorithms to identify patterns and relationships between entities, such as drugs, diseases, and symptoms, and provides users with relevant information and insights in a structured format.


# [microsoft/phi-2]
Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.

Solution:
A Knowledge Graph is a representation of relationships between entities, such as medical conditions, drugs, and patients. It helps healthcare professionals to identify and analyze complex patterns in large datasets, leading to improved diagnosis and treatment.

Follow-up Exercise 1:
How is a Knowledge Graph different from a traditional relational database?

Solution:
A Knowledge Graph differs from a traditional relational database by representing entities as nodes and relationships as edges. It allows for more complex relationships between entities and enables the analysis of these relationships to extract insights that may not be apparent in a traditional database.

# 1. Model Swap & Comparison

TinyLlama’s answer is richer in descriptive detail and demonstrates stronger correctness than phi-2's answer.  
In contrast, phi-2’s response is concise and well-formatted. However, it introduces itrrelevant content that was not part of the question.
Overall, TinyLlama returned a better answer because it stayed focused on the prompt, provided more relevant details, and avoided unnecessary information.

In [20]:
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids

print("Token count:", len(ids))
print("First 20 token IDs:", ids[:20])
print("Back to text:", tokenizer.decode(ids))

Token count: 16
First 20 token IDs: [1, 8218, 479, 17088, 3382, 1379, 508, 18195, 24609, 322, 19138, 675, 24899, 936, 11486, 29889]
Back to text: <s> Large Language Models can draft emails and summarize clinical notes.


In [21]:
base_prompt = "Give 3 shrot tips for writing reproducible data science code:"

settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.90, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.85, "top_k": 50},
]

for i, s in enumerate(settings, 1):
  t0 = time.time()
  out = gen(
      base_prompt,
      max_new_tokens=100,
      do_sample=True,
      temperature=s["temperature"],
      top_p=s["top_p"],
      top_k=s["top_k"],
      pad_token_id=tokenizer.eos_token_id,
  )[0]["generated_text"]
  print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
  print(out)
  print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 shrot tips for writing reproducible data science code: 1. Use comments to explain what each line of code does. 2. Use indentation to make your code easier to read. 3. Use functions to encapsulate your code and make it easier to reuse. 4. Use variables to store data and keep track of your work. 5. Use error handling to catch unexpected errors and provide clear feedback to the user.
(latency ~2.28s)

--- Variant 2 | temp=0.8 top_p=0.9 top_k=50 ---
Give 3 shrot tips for writing reproducible data science code: 1. Follow a consistent naming convention: Write your code using a consistent naming convention, such as PascalCase, CamelCase, or snake_case. 2. Minimize unnecessary variable names: Keep your code as minimal as possible by minimizing the number of variables and avoiding unnecessary variables. 3. Use descriptive variable names: Use descriptive variable names that clearly describe what the variable is doing. For example, instead 

# 2. Decoding Parameters – Explain in Your Own Words

Temperature controls randomness: low values (e.g., 0.2 in Variant 1) give safe, repetitive outputs, while high values (1.1 in Variant 3) create more diverse but less predictable text.
Top-p sets a probability threshold: high top-p (0.95 in Variant 1) considers more token options, while lower top-p (0.85 in Variant 3) narrows choices to safer words.
Top-k limits how many tokens are sampled: high k (50) allows more variety, low k forces the model to pick from fewer options.

In the output above, Variant 1 with low temperature(0.2) generated simple and basic tips that can apply to any coding environment. Variant 2 gave more practical tips with examples, and Variant 3 was concisel, and it gave more creatvie answer such as using reusable libraries whenever possible.
Use low temperature + higher top-p/k when you need accuracy and consistency (e.g., coding tips) and higher temperature + lower top-p/k for creativity or brainstorming.

# 3. Hallucinations - Risks & Mitigations

1. phi-2 Knowledge Graph Task: The model was asked to explain a Knowledge Graph in healthcare in three sentences. Instead, it added an unprompted follow-up exercise (“How is a Knowledge Graph different from a traditional relational database?”) along with a solution that was not requested.
2. Reproducible Data Science Tips Task: The base prompt asked for three short tips. However, Variant 1 generated five tips instead of three, and Variant 2 listed four tips but cut off mid-sentence, failing to follow instructions.

These hallucinations undermine reliability, especially in technical or academic contexts.  
By grounding outputs with external sources and adjusting generation parameters for tighter control, we can reduce irrelevant additions and maintain closer adherence to the user’s intent.  

In [7]:
def build_prompt(history, user_msg, system="Youu are a helpful data science assistant."):
  convo = [f"[SYSTEM] {system}"]
  for u, a in history[-3:]:
    convo.append(f"[USER] {u}")
    convo.append(f"[ASSISTANT] {a}")
  convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
  return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
  prompt = build_prompt(history, user_msg)
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  with torch.no_grad():
    t0 = time.time()
    tokens = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  text = tokenizer.decode(tokens[0], skip_special_tokens=True)
  reply = text.split("[ASSISTANT]")[-1].strip()
  history.append((user_msg, reply))
  print(reply)

# demo turns
chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

Transfer learning is a technique that allows a pre-trained deep learning model to be used for other tasks without having to train it from scratch. The pre-trained model has already learned from a large dataset, and it can be used to improve the performance of a new task.
When fine-tuning small LLMs on tiny datasets, there are two main risks: (1) Overfitting: small datasets can make the LLM's training set too small, resulting in a model that is too sensitive to training data. (2) Inference slowdown: small datasets may not be large enough to train a large LLM on, which can result in inference slowdown. To mitigate these risks, some approaches include using larger training sets, fine-tuning on multiple LLMs, and using larger batch sizes.
Certainly! LLMs are being used in various natural language processing applications,


In [8]:
import pandas as pd

prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]

rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9,
              pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})

df = pd.DataFrame(rows)
df


,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,1.82
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,0.04
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,2.84
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.,0.03


In [9]:
from google.colab import files

df.to_csv("outputs.csv", index=False, encoding="utf-8")
files.download("outputs.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>